# Analysis of Volatility Instruments

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import yfinance as yf

import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt


# Imports from internal packages
import sys

module_path1 = str(Path("../vixcoin_functions/"))
if module_path1 not in sys.path:
    sys.path.append(module_path1)
from feature_functions import (garch_fit_and_predict, 
                               correlation_filter, 
                               retrieve_yahoo_close, 
                               retrieve_yahoo_volume)

# Generation of the Features Matrix X

### X1: close prices
#### 40 units: 
    * Close prices of international indexes of stocks and bonds, 
    * key stocks, 
    * currency exchange rates, 
    * commodities 

In [22]:
# Ticker List: VIX must be in first position
# ticker_list_ORIGINAL= ["^VIX", "spy", 'XLF', 'XLE',
#               'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
#               '^TNX', 'ZB=F', 'ZF=F', 'NQ=F','NKD=F',                                       
#               'LQD',
#               'AAPL', 'AMZN', 'GE','MU','MSFT', 'BMY', 'FDX', 'GS','PLD','NVDA',   
#               "tlt", "ief", 
#               "FXI", "EZU", "EEM", "EFA", 'FEZ', "^GDAXI", '^FTSE','^HSI','^FCHI',              #'^GSPC',
#               "gld", "slv", "CL=F"]

ticker_list= ["^VIX", "VIXM", "VXX","UVXY", "spy"]



    
# Some of the less familiar tickers are listed below
# CAC 40 (^FCHI)
# Yen Denominated TOPIX Futures,D (TPY=F)
# FTSE 100 (^FTSE)
# SPDR EURO STOXX 50 ETF (FEZ)
# DAX PERFORMANCE-INDEX (^GDAXI)
# S&P500 Index (^GSPC)
# HANG SENG INDEX (^HSI)
# 13 Week Treasury Bill (^IRX) --eliminate
# Nikkei/USD Futures,Dec-2021 (NKD=F)
# iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
# Nasdaq 100 Dec 21 (NQ=F)
# NVIDIA Corporation (NVDA)
# Euro spot  'EURUSD'
# American Funds U.S. Government Securities Fund Class C (UGSCX) - 2001
# Micron Technology, Inc. (MU)
# Microsoft Corporation (MSFT)
# Bristol-Myers Squibb Company (BMY)
# FEDEX CORP (FDX)
# The Goldman Sachs Group, Inc. (GS)
# Prologis, Inc. (PLD)
# Treasury Yield 10 Years (^TNX) -- 1985
# Energy Select Sector SPDR Fund (XLE)
# Financial Select Sector SPDR Fund (XLF)
# U.S. Treasury Bond Futures,Dec- (ZB=F) - 2000
# Five-Year US Treasury Note Futu (ZF=F) - 2000
# E-Mini S&P500 (ES=F)
# E-Mini oil (QM=F)
# E-MinI Nasdaq (BIO=F)
print(f"Current number of tickers: {len(ticker_list)}")


Current number of tickers: 5


In [23]:
# X1: Upload of data using API
def retrieve_close(close_prices_dict, ticker_list):
    """
    This function retrieves close prices from Yahoo Finance
    
    Arg:
    close_prices_dict: empty dictionary to be filled with the close prices
    ticker_list: a list of tickers of the yahoo close prices to be retrieve
    
    Return:
    A dictionary with close prices
    """
    
    
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2011-02-01', end_date='2021-11-17')
        close_prices_dict[ticker] = close_price
    return close_prices_dict

close_prices_dict = {}
close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
close_prices_df= pd.DataFrame(close_prices_dict)

print("Completed retrieve of close prices")

Processing Close ^VIX
Processing Close VIXM
Processing Close VXX
Processing Close UVXY
Processing Close spy
Completed retrieve of close prices


In [24]:
returns=close_prices_df.pct_change()
returns["VIXM"].hvplot()

:Curve   [Date]   (VIXM)

In [25]:
#returns["^VIX"].hvplot()

In [26]:
returns.describe()

,^VIX,VIXM,VXX,UVXY,spy
count,2717.000000,2717.000000,960.000000,2547.000000,2717.000000
mean,0.003250,-0.000597,-0.000594,-0.004441,0.000604
std,0.084733,0.020836,0.049939,0.077811,0.010640
min,-0.269583,-0.140413,-0.155574,-0.334499,-0.109424
25%,-0.043936,-0.011963,-0.028254,-0.045922,-0.003379
50%,-0.006309,-0.001975,-0.008006,-0.011468,0.000728
75%,0.036630,0.008439,0.016054,0.025275,0.005453
max,1.155979,0.182147,0.370602,0.662064,0.090603


### We can see that 50% of the times the return of VIXM is negative, which is risky. Hopefully a ML model can help us.


In [27]:
returns_df=returns['VXX'].dropna()
returns_df

Date
2018-01-26    0.000000
2018-01-29    0.069414
2018-01-30    0.032792
2018-01-31    0.003273
2018-02-01   -0.050343
                ...   
2021-11-10    0.018388
2021-11-11   -0.021296
2021-11-12   -0.036897
2021-11-15   -0.011788
2021-11-16    0.003479
Name: VXX, Length: 960, dtype: float64

In [28]:
for i in range(0,5):
    print (f"Day of week {i}")
    print(returns_df[returns_df.index.dayofweek==i].describe())

Day of week 0
count    183.000000
mean       0.006187
std        0.062352
min       -0.155574
25%       -0.027225
50%       -0.008500
75%        0.022530
max        0.370602
Name: VXX, dtype: float64
Day of week 1
count    197.000000
mean       0.000165
std        0.041125
min       -0.085010
25%       -0.025982
50%       -0.000795
75%        0.019468
max        0.166418
Name: VXX, dtype: float64
Day of week 2
count    194.000000
mean      -0.002617
std        0.045821
min       -0.114447
25%       -0.027861
50%       -0.010398
75%        0.010620
max        0.179286
Name: VXX, dtype: float64
Day of week 3
count    194.000000
mean       0.001458
std        0.053916
min       -0.103733
25%       -0.024785
50%       -0.010460
75%        0.017750
max        0.336653
Name: VXX, dtype: float64
Day of week 4
count    192.000000
mean      -0.007866
std        0.043776
min       -0.117600
25%       -0.037044
50%       -0.011208
75%        0.014941
max        0.128594
Name: VXX, dtype: float64


In [29]:
returns.corr()

,^VIX,VIXM,VXX,UVXY,spy
^VIX,1.000000,0.755356,0.886350,0.883209,-0.746348
VIXM,0.755356,1.000000,0.888037,0.893715,-0.770187
VXX,0.886350,0.888037,1.000000,0.975780,-0.773541
UVXY,0.883209,0.893715,0.975780,1.000000,-0.773006
spy,-0.746348,-0.770187,-0.773541,-0.773006,1.000000


# Observations on correlations
The higher correlation with VIX is UVXY and VXX, which has short history. However, VXX has a high correlation with VIXM, which has a longer history. 
Then a good start is to invest in VIXM, and in the future use a similar model to invest in VXX and UVXY.  